In [1]:
import numpy as np

import os
from tqdm import tqdm
import gc
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
f'Tensorflow version: {tf.__version__}'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

'Tensorflow version: 2.12.0'

In [2]:
batch_size = 16
image_size = 480
samples = 4096

In [3]:
datasets = ["/kaggle/input/indian-birds-2/training_data_16", "/kaggle/input/indian-birds-2/training_data_17", "/kaggle/input/indian-birds-2/training_data_18", "/kaggle/input/indian-birds-2/training_data_19", "/kaggle/input/indian-birds-2/training_data_20"]
# datasets=[]
# datasets.extend(tf.io.gfile.glob("/kaggle/input/indian-birds-250/training_data_[^0-5]/*"))
print(len(datasets))
print(datasets)

5
['/kaggle/input/indian-birds-2/training_data_16', '/kaggle/input/indian-birds-2/training_data_17', '/kaggle/input/indian-birds-2/training_data_18', '/kaggle/input/indian-birds-2/training_data_19', '/kaggle/input/indian-birds-2/training_data_20']


In [4]:
train_ds = None
val_ds = None
class_names = []
increment = 750

for i,ds in enumerate(datasets):
    tds = image_dataset_from_directory(
        ds,
        seed=123,
        image_size=(image_size, image_size),
        batch_size=batch_size
    )

    class_names.extend(tds.class_names)

    tds = tds.map(lambda x, y: (x, y + increment))
    if train_ds is None:
        train_ds = tds
    else:
        train_ds = train_ds.concatenate(tds)

    if ds != datasets[-1]:
        increment += len(os.listdir(datasets[i+1]))

Found 40847 files belonging to 50 classes.
Found 44925 files belonging to 50 classes.
Found 33922 files belonging to 50 classes.
Found 40103 files belonging to 50 classes.
Found 35535 files belonging to 50 classes.


In [5]:
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def image_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()]))

def serialize_example(feature0, feature1):
    feature = {
      'feature0': image_feature(feature0),
      'feature1': int64_feature(feature1)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()
    
def tf_serialize_example(f0,f1):
    tf_string = tf.py_function(serialize_example, (f0, f1), tf.string)
    return tf.reshape(tf_string, ())

In [6]:
if not os.path.exists('tfrecords'):
    os.mkdir('tfrecords')

In [7]:
class_names

['American Pipit',
 'Amur Paradise-Flycatcher',
 'Amur Stonechat',
 'Arctic Warbler',
 'Ashambu Laughingthrush',
 'Ashy Minivet',
 'Asian Desert Warbler',
 'Asian Dowitcher',
 'Asian Stubtail',
 'Australasian Grass-Owl',
 'Baer_s Pochard',
 'Baikal Bush Warbler',
 'Baikal Teal',
 'Baillon_s Crake',
 'Banasura Laughingthrush',
 'Bar-tailed Godwit',
 'Beach Thick-knee',
 'Bimaculated Lark',
 'Black-bellied Sandgrouse',
 'Black-bellied Tern',
 'Black-browed Reed Warbler',
 'Black-faced Bunting',
 'Black-headed Bunting',
 'Black-rumped Magpie',
 'Black-throated Accentor',
 'Black-throated Thrush',
 'Blue-and-white Flycatcher',
 'Blue-winged Pitta',
 'Blunt-winged Warbler',
 'Blyth_s Pipit',
 'Booted Warbler',
 'Brambling',
 'Broad-billed Sandpiper',
 'Brown-breasted Flycatcher',
 'Brown-capped Laughingthrush',
 'Brown-cheeked Rail',
 'Brown-rumped Minivet',
 'Burmese Shrike',
 'Caspian Gull',
 'Caspian Plover',
 'Cetti_s Warbler',
 'Chestnut Bunting',
 'Chestnut-cheeked Starling',
 'Chestn

In [8]:
with open('cnames.txt','w') as f:
    f.write(str(class_names))

In [9]:
train_ds = train_ds.unbatch()

In [10]:
del tds,increment,datasets,ds
gc.collect()

83

In [11]:
serialized=list()

for images, labels in tqdm(train_ds):
    serialized.append(serialize_example(images.numpy(),labels.numpy()))

195332it [34:16, 94.97it/s] 


In [12]:
gc.collect()

0

In [13]:
l = len(serialized)

In [14]:
del f,images,labels
gc.collect()

0

In [15]:
ix = l//samples
serialized_temp = list()
for i in range(ix):
    serialized_temp.append(serialized[i*samples:(i+1)*samples])
serialized_temp.append(serialized[(ix)*samples:])

In [16]:
del serialized, ix

In [17]:
%%time
for i,s in enumerate(serialized_temp):
    filename = os.path.join("tfrecords","batch_3_{:02.0f}.tfrec".format(i))
    with tf.io.TFRecordWriter(filename) as writer:
        for rec in s:
            writer.write(rec)
del i,s,serialized_temp,writer
gc.collect()

CPU times: user 11.6 s, sys: 17.2 s, total: 28.8 s
Wall time: 59.1 s


0